In [3]:
from datetime import datetime, timedelta
import os
import subprocess

table = "nn"

In [118]:
q = '''SELECT   scaled,   course,   tagblock_station,   cs,   speed,   unit,   gridcode,  
distance_from_port,   lc,   nnet_score,   lon,   commstate_cs_fill,   type,   distance_from_shore, 
timestamp(timestamp) timestamp,   band,   lat,   mode,   heading,   slot_timeout,   tagblock_T,  
spare2,   second,   sync_state,   tagblock_timestamp,   regions,   dsc,   msg22,   accuracy,   status, 
maneuver,   repeat,   tagblock_q,   mmsi,   raim,   spare,   device,   measure_new_score,   class, 
display,   turn,   seg_id,   slot_offset,   status_text FROM   [world-fishing-827:scratch_david.nn_allyears_20170411]
'''
# make the table partitioned

In [4]:
def divide_table(startdate_enddate):
    s = startdate_enddate.split("_")
    startdate = datetime.strptime(s[0],"%Y%m%d")
    enddate = datetime.strptime(s[1],"%Y%m%d")
    num_days = (enddate-startdate).days
    half = int(num_days / 2)
    middate = startdate + timedelta(days=half)
#     print startdate, middate, enddate
    startdate_formatted = startdate.strftime("%Y-%m-%d")
    middate_formatted = (middate+timedelta(days=1)).strftime("%Y-%m-%d")
    q = '''select * from [scratch_david.{table}__{startdate_enddate}] where \
    timestamp < timestamp("{middate_formatted}")'''.format(\
         table = table ,startdate_enddate=startdate_enddate, 
        middate_formatted = middate_formatted)    
    
    
    if s[0] == middate.strftime("%Y%m%d"):
        out_table =  table+"_"+s[0]
    else:
        out_table =  table+"__"+s[0]+"_"+middate.strftime("%Y%m%d")  
    command1 = "bq query  --replace=TRUE \
--destination_table=scratch_david.{out_table} --allow_large_results '{q}'".format(out_table = out_table, q=q)
#     print command
#     os.system(command)
    
    middate2 = middate + timedelta(days=1)   
    q = '''select * from [scratch_david.{table}__{startdate_enddate}] where \
    timestamp >= timestamp("{middate_formatted}")'''.format(\
         table = table ,startdate_enddate=startdate_enddate, 
        middate_formatted = middate_formatted)    
    
    
    if middate2.strftime("%Y%m%d") == s[1]:
        out_table = table+"_"+s[1]
    else:
        out_table = table+"__"+middate2.strftime("%Y%m%d")+"_"+s[1]
    command2 = "bq query  --replace=TRUE \
--destination_table=scratch_david.{out_table} --allow_large_results '{q}'".format(out_table = out_table, q=q)
#     print command
#     os.system(command)
    return [command1, command2]


In [6]:
for i in range(1):
    proc = subprocess.Popen(["bq ls --max_results 20000 scratch_david"], stdout=subprocess.PIPE, shell=True)
    (out, err) = proc.communicate()
    toprocess = out
    tp2 = toprocess.split("\n")
    tp2 = [t.replace(" ","").replace("TABLE","") for t in tp2]

    # get the tables that already exist
    tp3 = []
    for t in tp2:
        if t[:4] == "nn__":
            tp3.append(t)

    commands = []
    for t in tp3:
        commands += divide_table(t[4:])

    # use gnu parallel to run all the queries

    with open('commands.txt', 'w') as f:
        f.write("\n".join(commands))

    os.system("parallel -j 16 < commands.txt")
    os.system("rm -f remove_commands.txt")

    # delete the old tables
    commands = []
    for t in tp3:
        commands.append("bq rm -f scratch_david."+t)

    with open('commands.txt', 'w') as f:
        f.write("\n".join(commands))

    os.system("parallel -j 16 < commands.txt")
    os.system("rm -f remove_commands.txt")
    print "Number of tables on this round:",len(commands)


Number of tables on this round: 0
